# Import Libraries

In [101]:
# import all the required packages
import sys
import os
# change according to the status of GPU
# this command must before import pytorch
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # setting the GPUs

In [102]:
import pandas as pd
import numpy as np
import json, csv

import torch
from datasets import load_dataset, load_metric
from transformers import AutoModel, BertForSequenceClassification, BertTokenizer
from sklearn.preprocessing import LabelEncoder

In [103]:
# check the running environment
print(sys.executable)
print(torch.cuda.is_available())
torch.cuda.device_count()

/home/user/jinqli/anaconda3/envs/new_env/bin/python
True


1

# Load jobads data

In [104]:
#data_path = '/srv/scratch2/jinq/taskontology/jobad_taskdata/task_sample-0.2.jsonl'
#jobads = pd.read_json(data_path,lines=True)
#print('numbers of records in job ads:', jobads.shape[0])
#jobtask = jobads[jobads['label']=='TASK']
#jobtask

gold_path = '/srv/scratch2/jinq/taskontology/jobad_taskdata/goldstandard-subspans-ActObj.jsonl'
goldCont_path = '/srv/scratch2/jinq/taskontology/jobad_taskdata/goldstandard-subspans-ActObjCont.jsonl'

gold_obj = pd.read_json(gold_path,lines=True)
gold_objCont = pd.read_json(goldCont_path,lines=True)

print('numbers of records in job ads:', gold_objCont.shape[0])
gold_obj

numbers of records in job ads: 4586


,id,meta,text,label
0,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Verpackungsarbeiten,ActObj
1,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Einrichten des eigenen Arbeitsplatzes,ActObj
2,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Kontrollarbeiten,ActObj
3,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Aussenarbeit,ActObj
4,sjmm-22011108302017,"{'id': 'sjmm-22011108302017', 'year': 2011, 'i...","Planung, Durchführung und Auswertung von hocha...",ActObj
...,...,...,...,...
4581,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",realisierst du herausfordernde Automatisierungen,ActObj
4582,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",Entwicklung einer Self-Service Plattform,ActObj
4583,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...","Weiterentwicklung, Optimierung und Erneuerung ...",ActObj
4584,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",Implementierung von Continuous Deployment Prin...,ActObj


In [105]:

model_checkpoint = 'trained_models/multibert_results/checkpoint-1000/'

# import the tokenizer 
SPECIAL_TOKENS_MAP_FILE = 'special_tokens_map.json'
TOKENIZER_CONFIG_FILE = 'tokenizer_config.json'

# import the model
model = BertForSequenceClassification.from_pretrained(model_checkpoint, problem_type="multi_label_classification")
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

In [106]:
print(model.num_labels)

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

37


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [32]:
# example
inputs = tokenizer(gold_obj['text'].tolist()[0], padding=True, truncation=True, return_tensors="pt")
print(inputs)

# encoded all the text of jobads
encoded_data = [tokenizer(item['text'], 
                              return_tensors="pt", padding=True, truncation=True) for _,item in gold_obj.iterrows()]

print(encoded_data[0])

{'input_ids': tensor([[   101,  41357,  78560,  32346, 106172,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[   101,  41357,  78560,  32346, 106172,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [107]:
max_length = 128
def get_prediction(text, top_n: int=5):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    # perform inference to our model
    logits = model(**inputs).logits
    # get output probabilities by doing softmax
    # probs = outputs[0].softmax(1)
    
    #get the top_n candidates and corresponding logits as score
    value, indices = logits.topk(top_n, 1,sorted=True)
    results = [(id_.item(),round(val.item(),4)) for val,id_ in zip(value[0], indices[0])]
    results = [j for item in results for j in item ]
    return results

def apply_classify_on_df(df):
    """
    Apply a function and return multiple values so that you can create multiple columns, return a pd.Series with the values instead:
    Source: https://queirozf.com/entries/pandas-dataframes-apply-examples
    """
    df[['la1', 'la1score', 'la2', 'la2score', 'la3', 'la3score', 'la4', 'la4score', 'la5', 'la5score']] = df.apply(lambda row: pd.Series(get_prediction(row.loc['text'])), axis=1)
    return df

In [108]:
# one example
get_prediction(gold_obj['text'].tolist()[0])

[13, 3.5669, 20, 2.972, 18, 2.2326, 33, 1.9773, 7, 1.4851]

In [109]:
predicted_gold = apply_classify_on_df(gold_obj)
predicted_gold.head()

,id,meta,text,label,la1,la1score,la2,la2score,la3,la3score,la4,la4score,la5,la5score
0,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Verpackungsarbeiten,ActObj,13.0,3.5669,20.0,2.9720,18.0,2.2326,33.0,1.9773,7.0,1.4851
1,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Einrichten des eigenen Arbeitsplatzes,ActObj,34.0,4.4625,12.0,3.5066,32.0,1.8612,17.0,1.2804,30.0,0.6507
2,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Kontrollarbeiten,ActObj,5.0,2.7041,21.0,2.7031,24.0,2.3517,23.0,1.1801,28.0,1.0396
3,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Aussenarbeit,ActObj,28.0,3.7490,24.0,2.8133,20.0,2.5260,13.0,1.7691,7.0,1.2127
4,sjmm-22011108302017,"{'id': 'sjmm-22011108302017', 'year': 2011, 'i...","Planung, Durchführung und Auswertung von hocha...",ActObj,15.0,4.3477,9.0,2.9791,19.0,2.1822,0.0,1.5102,13.0,1.2973


In [97]:
predicted_goldCon = apply_classify_on_df(gold_objCont)
predicted_goldCon.head()

,id,meta,text,label,la1,la1score,la2,la2score,la3,la3score,la4,la4score,la5,la5score
0,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Verpackungsarbeiten allgemein und in Medizinal...,ActObjCont,13.0,5.5087,24.0,4.1929,28.0,2.8080,5.0,2.4437,20.0,2.1681
1,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Einrichten des eigenen Arbeitsplatzes,ActObjCont,34.0,4.4625,12.0,3.5066,32.0,1.8612,17.0,1.2804,30.0,0.6507
2,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Kontrollarbeiten,ActObjCont,5.0,2.7041,21.0,2.7031,24.0,2.3517,23.0,1.1801,28.0,1.0396
3,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Aussenarbeit,ActObjCont,28.0,3.7490,24.0,2.8133,20.0,2.5260,13.0,1.7691,7.0,1.2127
4,sjmm-22011108302017,"{'id': 'sjmm-22011108302017', 'year': 2011, 'i...","Planung, Durchführung und Auswertung von hocha...",ActObjCont,15.0,4.3022,9.0,2.9254,0.0,2.3347,19.0,2.1692,11.0,2.0008


## Inverse GWA class to title

In [117]:
predicted_gold['la1'].

0       13.0
1       34.0
2        5.0
3       28.0
4       15.0
        ... 
4581    33.0
4582    33.0
4583    36.0
4584    33.0
4585    12.0
Name: la1, Length: 4586, dtype: float64

In [123]:
de_path = '/srv/scratch2/jinq/taskontology/task_to_GWA_IWA_DWA_DE.csv'
dataset = load_dataset('csv', data_files=de_path,split='train')

# encoding labels
y_encoded = LabelEncoder().fit(dataset['GWA Title'])

gwa_titles = y_encoded.inverse_transform(predicted_gold['la1'].astype(int)).tolist()
print(len(gwa_titles))


Using custom data configuration default-2bd0bdea745cdb59
Found cached dataset csv (/home/user/jinqli/.cache/huggingface/datasets/csv/default-2bd0bdea745cdb59/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


4586


In [124]:
predicted_gold['gwa_class'] = gwa_titles
predicted_gold

,id,meta,text,label,la1,la1score,la2,la2score,la3,la3score,la4,la4score,la5,la5score,gwa_class
0,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Verpackungsarbeiten,ActObj,13.0,3.5669,20.0,2.9720,18.0,2.2326,33.0,1.9773,7.0,1.4851,Handling and Moving Objects
1,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Einrichten des eigenen Arbeitsplatzes,ActObj,34.0,4.4625,12.0,3.5066,32.0,1.8612,17.0,1.2804,30.0,0.6507,Training and Teaching Others
2,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Kontrollarbeiten,ActObj,5.0,2.7041,21.0,2.7031,24.0,2.3517,23.0,1.1801,28.0,1.0396,Controlling Machines and Processes
3,sjmm-22011109412481,"{'id': 'sjmm-22011109412481', 'year': 2011, 'i...",Aussenarbeit,ActObj,28.0,3.7490,24.0,2.8133,20.0,2.5260,13.0,1.7691,7.0,1.2127,Repairing and Maintaining Mechanical Equipment
4,sjmm-22011108302017,"{'id': 'sjmm-22011108302017', 'year': 2011, 'i...","Planung, Durchführung und Auswertung von hocha...",ActObj,15.0,4.3477,9.0,2.9791,19.0,2.1822,0.0,1.5102,13.0,1.2973,"Inspecting Equipment, Structures, or Materials"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4581,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",realisierst du herausfordernde Automatisierungen,ActObj,33.0,4.0585,6.0,2.6155,18.0,2.3580,17.0,1.6129,36.0,1.3297,Thinking Creatively
4582,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",Entwicklung einer Self-Service Plattform,ActObj,33.0,4.2973,36.0,3.0622,6.0,1.8823,5.0,0.7286,7.0,0.7216,Thinking Creatively
4583,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...","Weiterentwicklung, Optimierung und Erneuerung ...",ActObj,36.0,3.8969,33.0,2.5566,6.0,1.3290,5.0,0.9453,13.0,0.4937,Working with Computers
4584,sjmm_large-9202135719010297,"{'id': 'sjmm_large-9202135719010297', 'year': ...",Implementierung von Continuous Deployment Prin...,ActObj,33.0,3.1539,36.0,2.8204,18.0,2.3654,6.0,2.2794,14.0,0.5425,Thinking Creatively
